In [ ]:
from collections import defaultdict
from mysql.connector.errors import Error
from itertools import islice

import datetime
import sys
import json
import mysql.connector
import os
import glob
import re


In [3]:
d=defaultdict(list)
fileslist=[]
errorlist=[
          '/media/data/sensemaking/dataset/Vansterpartiet/*.dat',
          '/media/data/sensemaking/dataset/Sveriagedemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Socialdemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Piratpartiet/*.dat',
          ]

dirlist= ['/media/data/sensemaking/dataset/centralpartiet/clean/*.dat',
          '/media/data/sensemaking/dataset/FeministisktInitiativ/clean/*.dat',
          '/media/data/sensemaking/dataset/Kristdemokraterna/clean/*.dat',
          '/media/data/sensemaking/dataset/Liberalerna/clean/*.dat',
          '/media/data/sensemaking/dataset/Vansterpartiet/*.dat',
          '/media/data/sensemaking/dataset/Sveriagedemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Socialdemokraterna/*.dat',
          '/media/data/sensemaking/dataset/Piratpartiet/*.dat'
         ]

for dirl in dirlist:
    fileslist.extend(glob.glob(dirl,recursive=True))
    print(re.search('dataset/(.+?)/',dirl).group(1))
    print('directory files are adding to filelist: %s:'%dirl)
print('no.of file in directory: %d'%len(fileslist))

    

centralpartiet
directory files are adding to filelist: /media/data/sensemaking/dataset/centralpartiet/clean/*.dat:
FeministisktInitiativ
directory files are adding to filelist: /media/data/sensemaking/dataset/FeministisktInitiativ/clean/*.dat:
Kristdemokraterna
directory files are adding to filelist: /media/data/sensemaking/dataset/Kristdemokraterna/clean/*.dat:
Liberalerna
directory files are adding to filelist: /media/data/sensemaking/dataset/Liberalerna/clean/*.dat:
Vansterpartiet
directory files are adding to filelist: /media/data/sensemaking/dataset/Vansterpartiet/*.dat:
Sveriagedemokraterna
directory files are adding to filelist: /media/data/sensemaking/dataset/Sveriagedemokraterna/*.dat:
Socialdemokraterna
directory files are adding to filelist: /media/data/sensemaking/dataset/Socialdemokraterna/*.dat:
Piratpartiet
directory files are adding to filelist: /media/data/sensemaking/dataset/Piratpartiet/*.dat:
no.of file in directory: 142


In [ ]:

user=[]
tweet=[]
mentions=[]
mentions_party=[]
retweet=[]
hashtaglist=[]
hashtag_nodup_list=[]
tweet_hashtag_list=[]
tweet_hash_list=[]
tweet_url_list=[]
print(len(user))
# each file listed in fileslist 
for file in fileslist:
    #print(file)
    with open(file) as f:
        content = f.readlines()
         # you may also want to remove whitespace characters like `\n` at the end of each line
    #content = [x.strip() for x in content]
    #print(len(content))
       
    for x in content:
           
        try:
            if x.strip().endswith('}'):
                jsondata = json.loads(x,encoding='latin1')
            else:
                
                continue
                #   jsondata = json.loads(x,encoding='latin1')
                    
        except ValueError as ex:
            #print(f)
           
            continue
            #raise ValueError(ex)
        #tweet-table data    
        tweet_id= jsondata['id']
        text= jsondata['text']
        retweet_count=jsondata['retweet_count']
        in_reply_to_user_id= jsondata['in_reply_to_user_id_str']
        in_reply_to_status_id= jsondata['in_reply_to_status_id_str']
        tweet_created=datetime.datetime.strptime(jsondata['created_at'], "%a %b %d %H:%M:%S +0000 %Y") 
        date1= (tweet_created).date()
        time1= (tweet_created).time()
        tweet_created_at= datetime.datetime.combine(date1,time1)
        #mentions-party table-data
        party_name=re.search('dataset/(.+?)/',file).group(1)
        
        

        # user table data

        userdata= jsondata['user']
        id_str=  int(userdata['id_str'])
           
        if userdata['verified']:
            verified=1
        else:
            verified=0              
        followers_count= userdata['followers_count']
        
        id=userdata['id']
        description= userdata['description']
       
        friends_count=userdata['friends_count']
        location=userdata['location']
        
        created_at_date= datetime.datetime.strptime(userdata['created_at'], "%a %b %d %H:%M:%S +0000 %Y")
        date= (created_at_date).date()
        time= (created_at_date).time()
        created_at= datetime.datetime.combine(date,time)
        name=userdata['name']
       
        
        #1. retweet table realted
        
        if jsondata.get('retweeted_status'):
            #retweet table data
            retweet_data= jsondata['retweeted_status']
            tweet_id_old= retweet_data['id']
           # retweet.append((tweet_id,tweet_id_old))
        #Hahtag table related
        enitites=jsondata['entities']
        
        hashtags=enitites['hashtags']
        if len(hashtags) :
            hashtag_list=[]
            for i in hashtags:
                
                hashtag=i['text']
                hashtag_list.append(hashtag)
                hashtaglist.append(hashtag.lower())
                
        tweet_hashtag_list.append((tweet_id,hashtag_list))
        # urls tables data
        urls=enitites['urls']
        if len(urls):               
                for i in urls:
                    url=i['url']
                    tweet_url_list.append((tweet_id,url))                    
        else: 
             tweet_url_list.append((tweet_id,'null'))
        #tweet.append((tweet_id,[text,id_str,retweet_count,tweet_created_at,in_reply_to_user_id,in_reply_to_status_id]))

        
                
                
        #print( tweet_hashtag_list)
        #uncomment the below line if user data to dump inot mysql
        
        #user.append((id_str,[verified,followers_count,id,description, friends_count, location, created_at,name]))
        #mentions.append((tweet_id,id_str))
        #mentions_party.append((tweet_id,party_name))
       
        
        
        #print(id_str)
        #print(len(mentions_party))
"""
print(len(tweet_url_list))
print(tweet_url_list[0])
print(tweet_url_list[1])
print(tweet_url_list[2])
        
print(len(hashtaglist)) 
#list1=[['a','b'],['b','c']]-->{'b', 'a', 'c'}
"""
hashtag_set=set(hashtaglist)
for  i in hashtag_set:
    hashtag_nodup_list.append((i,datetime.datetime.now(),datetime.datetime.now()))
print(len(hashtag_nodup_list))

#list2=[(1,[2,3]),(2,[3,4])] -->[(1, 2), (1, 3), (2, 3), (2, 4)]

for k,v in tweet_hashtag_list:
    for i in v:
        tweet_hash_list.append((k,i))
                               
print(len(tweet_hash_list))        
#print(tweet_hash_list[0])
#print(tweet_hash_list[1])
#print(tweet_hash_list[2])
                             
                               

#print (len(user))
#print(len(tweet))
#print(len(mentions_party))
#print(len(retweet))
                               
print(len(tweet_hashtag_list))
                               
#for i=5 in retweet:
#print(retweet[0])
#print(retweet[1])
    
#s1=userinsertsql(user)
#s1= tweetinsertsql(tweet)
#s1=mentionsinsertsql(mentions)
#s1=mentionsPartyinsertsql(mentions_party)
#s1= retweetinsertsql(retweet)
#s1= hashtaginsertsql(hashtag_nodup_list)
                               
s1= tweet_hashtaginsertsql(tweet_hash_list)
#s1= tweet_url_insertsql(tweet_url_list)
print('user table insertion operation : %s'%s1)

In [7]:
def userinsertsql(user):
   
    user_list=[]
    print('user duplicates: %d'%len(user))
    # to eliminate duplicate entrys in "user" list 
    user_noduplicate =dict(user)
    print('user no dplicates: %d'%len(user_noduplicate))
    # to convert  [k,[v]]  into  a  list [k,v]
    for k,v in user_noduplicate.items():
        user_list.append(tuple([k]+v))
    print('user no dplicates: %d'%len(user_list))
    
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    #add_user= "INSERT INTO users(id_str,verified, followers_count, id, description,friends_count, location,created_at, name) VALUES (?, ?, ?, ?, ?, ? , ?, ?, ?)"
    
    #for x in user_list]
    #print(len(user_list))
    
    #print(user_list)
    try:
        cursor = cnx.cursor(prepared=True)                        
        #cursor.executemany(add_user,user_list)
        #cursor.execute(add_user,temp)
        for i in user_list:
            cursor.execute("update users set location=? where id_str=?",(i[6],i[0]))
            
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
    return 'success';


In [10]:
def tweetinsertsql(tweet):
    
    tweet_list=[]
    tweet_update_list=[]
    print(len(tweet))
    tweet_noduplicate =dict(tweet)
    print('tweet with out duplicates: %d'%len(tweet_noduplicate))
    
    for k,v in tweet_noduplicate.items():
        tweet_list.append(tuple([k]+v))
    print('user no dplicates: %d'%len(tweet_list))
    
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_tweet= "INSERT INTO tweet(tweet_id, text, id_str, retweet_count, created_at, in_reply_to_user_id, in_reply_to_status_id) VALUES (?, ?, ?, ?,?,?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        #cursor.executemany(add_tweet,tweet_list)
        for i in tweet_list:
            #cursor.execute("update tweet set created_at=? where tweet_id=?",(i[4],i[0]))
            #cursor.execute("update tweet set text=? where tweet_id=?",(i[1],i[0]))
            cursor.execute("update tweet set d= dayofyear(?),m= month(?),w=weekofyear(?) where tweet_id=?",(i[4],i[4],i[4],i[0]))
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';

In [ ]:
def mentionsinsertsql(mentions):
    
    mentions_list=[]
    print(len(mentions))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_mentions= "INSERT INTO mentions(tweet_id, user_id_str) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_mentions,mentions)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';

In [ ]:
def mentionsPartyinsertsql(mentions_party):
    
    mentions_partylist=[]
    print(len(mentions_party))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_mentions_party= "INSERT INTO mentions_party(tweet_id, party_name) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_mentions_party,mentions_party)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';

In [ ]:
def retweetinsertsql(retweet):
    
    retweet_list=[]
    print(len(retweet))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_retweet= "INSERT INTO retweet(tweet_id_new, tweet_id_old ) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_retweet,retweet)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';

In [64]:
def hashtaginsertsql(hashtag_nodup_list):
    
   
    print(len(hashtag_nodup_list))
       
    hash_text= []    
    hash_text_concat= []
    
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_hashtag= "INSERT INTO hashtag(text,first_usage,last_usage) VALUES (?, ?,?)" 
    #q="insert into dummy(text,f_usage,l_usage) select ?,min(created_at) as f_usage1, max(created_at) as l_usage1 from tweet join tweets_hashtag using(tweet_id) join hashtag on hashtag.hashtag_id=tweets_hashtag.hash_id where tweet.text like ?"
    q="insert into dummy(text,f_usage,l_usage) select ?,min(created_at) as f_usage1, max(created_at) as l_usage1 from tweet left join tweets_hashtag using(tweet_id) left join hashtag on hashtag.hashtag_id=tweets_hashtag.hash_id where hashtag.text=?"
    q1="update hashtag ht inner join dummy as t using(text) set first_usage=f_usage,last_usage=l_usage where text=?"
    print(q1)
    #update hashtag first_usage
    for i in hashtag_nodup_list:
        hash_text_concat.append(str('%'+i[0]+'%'))
        hash_text.append(str(i[0]))
    print(len(hash_text))
    for i in range(0,5):
        print(hash_text[i])
        print(hash_text_concat[i])
    
    """
    hash_id= (range(1,len(hashtag_nodup_list)+1))
    print(len(hash_id))
    for i in hashtag_nodup_list:
        hash_text.append(i[0])
    print(len(hash_text))
    hash_text_tuple=tuple(hash_text)
    print(len(hash_text_tuple))
    
    update_tuple=list(zip(hash_text_tuple,hash_id))
    print(len(update_tuple))
    print(update_tuple[0:10])
    update hashtag ht,(select min(tweet.created_at) as first_usage  from tweet, hashtag, tweets_hashtag where tweet.tweet_id = tweets_hashtag.tweet_id and hashtag.hashtag_id=tweets_hashtag.hash_id and tweet.text like '%#trump%') a1,(select max(tweet.created_at) as last_usage  from tweet, hashtag, tweets_hashtag where tweet.tweet_id = tweets_hashtag.tweet_id and hashtag.hashtag_id=tweets_hashtag.hash_id and tweet.text like '%#trump%')a2
 set ht.first_usage=a1.first_usage,ht.last_usage=a2.last_usage where ht.text='trump';
   """
    try:
        cursor = cnx.cursor(prepared=True)                        
        #cursor.executemany(add_hashtag,hashtag_nodup_list)
        #cursor.executemany(q.encode('latin-1'),hash_text)
        for i in range(len(hashtag_nodup_list)):
            
            #cursor.execute(q,(hash_text[i],hash_text[i]))
            cursor.execute(q1,(hash_text[i],))
            print(i)
        #for i in range(len(hashtag_nodup_list)):
            #cursor.execute("update hashtag ht,(select min(tweet.created_at) as first_usage  from tweet, hashtag, tweets_hashtag where tweet.tweet_id = tweets_hashtag.tweet_id and hashtag.hashtag_id=tweets_hashtag.hash_id and tweet.text like ?) a1 set ht.first_usage=a1.first_usage where ht.text=?",(hash_text_concat[i],hash_text[i]))
            #print(i)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
   
    return 'success';

In [16]:
def tweet_hashtaginsertsql(tweet_hashtag_list):
    
   
    print(len(tweet_hashtag_list))
    for i in range(5):
        print(tweet_hashtag_list[i])
    tweet_hashtag_list_nodup=list(set(tweet_hashtag_list))
    print(len(tweet_hashtag_list_nodup))
    for i in range(5):
        print(tweet_hashtag_list_nodup[i])
    
    hashtag=[] 
    tweet_hashtag=[]
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    #select_hash="select  hashtag_id, text from  hashtag"
    add_tweet_hashtag= "INSERT INTO tweets_hashtag(tweet_id,hash_id) select ?,hashtag_id from hashtag where text=?" 
    
    try:
        cursor = cnx.cursor(prepared=True) 
        """
        cursor.execute(select_hash)
        for (hashtag_id, text) in cursor:
            hashtag.append((hashtag_id, text.decode('utf-8')))
        hashtag_dict=dict([(t[1], t[0]) for t in hashtag])
       
        for i in tweet_hashtag_list_nodup:
            hash_id=hashtag_dict.get(i[1])
            tweet_hashtag.append((i[0],hash_id))
        print(len(tweet_hashtag))
       """
        for i in range(len(tweet_hashtag_list_nodup)):
            print(i)
            try:
                    cursor.execute(add_tweet_hashtag,(tweet_hashtag_list_nodup[i][0],tweet_hashtag_list_nodup[i][1]))
            except Error as err:
                print("Something went wrong: {}")
                print(i)
        
        cnx.commit()
        cursor.close()
        cnx.close() 
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
       
    return 'success';

In [8]:
def tweet_url_insertsql(tweet_url_list):
    
    tweet_list_url= []
    print(len(tweet_url_list))
    #tweet_list_nodup =dict(tweet_url_list)
    """
    i=1
    for k, v in d.items():
        if i<3:
            print(k, v)
            i+=1
        
    print('tweet with out duplicates: %d'%len(tweet_list_nodup))
    
    for k,v in tweet_list_nodup.items():
        tweet_list_url.append((k,v)) 
    print('tweet with out duplicates: %d'%len(tweet_list_url))
    print(tweet_list_url[0])
    print(tweet_list_url[1])
    print(tweet_list_url[2])
   """
    tweet_url_set=set(tweet_url_list)
    for i in tweet_url_set:
        tweet_list_url.append(i)
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_url= "INSERT INTO urls(tweet_id, url) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_url,tweet_list_url)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';